<a href="https://colab.research.google.com/github/andrybrew/IHT-SEM1302-30Okt/blob/main/practice_material/topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Topic Modeling**

##**Importing required libraries**

In [1]:
!pip install pyLDAvis --no-deps
!pip install funcy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.9 MB/s eta 0:00:00


In [32]:
import pandas as pd
import nltk
import requests
import seaborn as sns
import string
import gensim
import spacy
import re

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora
import pyLDAvis.gensim_models

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

##**Import Dataset from Apify Runs**

In [61]:
# Fetching the dataset from Apify
api_url = "https://api.apify.com/v2/datasets/na5aYoW2Rcrm9h08r/items"
api_token = "apify_api_NbL3VeatZbemul8vsNCyVKRhzmRv0M0Wh9ql"  # Substitute with your actual API token


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [62]:
# Making a GET request to the API
params = {"token": api_token, "format": "json"}  # parameter untuk permintaan
response = requests.get(api_url, params=params)

# Checking the response status
if response.status_code == 200:  # Sukses
    # Processing the JSON data into a pandas DataFrame
    data = response.json()
    df_tweet = pd.DataFrame(data)

    # Saving the DataFrame to a CSV file
    df_tweet.to_csv("twitter_data.csv", index=False)
    print("Data berhasil diunduh dan disimpan sebagai twitter_data.csv")
else:
    print(f"Terjadi kesalahan: {response.status_code}. Pesan: {response.text}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Data berhasil diunduh dan disimpan sebagai twitter_data.csv


##**Data Preprocessing for Topic Modeling**

In [63]:
# Function for removing URL
def remove_url(text):
    return re.sub(r'https?://\S+|www\.\S+|\S+\.\S+/\S+', '', text, flags=re.MULTILINE)

# Menghapus URL dari setiap tweet
df_tweet['text'] = df_tweet['text'].apply(remove_url)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [64]:
# Select News Title
tweet = df_tweet['text']

# Tokenize
tokenized_text = [d.lower().split() for d in tweet]

# Remove punctuation
punctuation = string.punctuation
tokenized_text = [[word for word in doc if word not in punctuation] for doc in tokenized_text]

# Lemmatization
lemmatizer = WordNetLemmatizer()
tokenized_text = [[lemmatizer.lemmatize(word) for word in doc] for doc in tokenized_text]

# Remove stopwords
stop_words = stopwords.words('indonesian')
tokenized_text = [[word for word in doc if word not in stop_words] for doc in tokenized_text]

# Adding additional words to the stop words list
custom_stop_words = ['dgn', 'sdh', 'yg', 'the', 'gak', 'ga', 'a', 'krn', 'thd', 'nya', 'ya', 'n', 'kalo', 'aja', 'deh', 'tuh', 'udah', 'dll.', '2', '25', '20', '1.', '2.', '7.', 'u', '5', 'gua', '•']
stop_words.extend(custom_stop_words)

# Remove stopwords again
tokenized_text = [[word for word in doc if word not in stop_words] for doc in tokenized_text]

# Create dictionary
dictionary = corpora.Dictionary(tokenized_text)

# Create corpus
corpus = [dictionary.doc2bow(doc) for doc in tokenized_text]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##**Setting Up LDA Model**

In [65]:
# Train LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=3,
                                           passes = 200,
                                           per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##**Visualizing Topics**

In [66]:
# Enable Notebook
pyLDAvis.enable_notebook()

# Visualize
pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.116240 -0.006208       1        1  51.154883
0      0.064698 -0.079172       2        1  26.810108
1      0.051542  0.085381       3        1  22.035009, topic_info=        Term       Freq      Total Category  logprob  loglift
740   persen  30.000000  30.000000  Default  30.0000  30.0000
423    tukar  14.000000  14.000000  Default  29.0000  29.0000
415    nilai  16.000000  16.000000  Default  28.0000  28.0000
95        6%  39.000000  39.000000  Default  27.0000  27.0000
621  naikkan  38.000000  38.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
27      bank   5.071035  69.773018   Topic3  -5.7534  -1.1092
82     orang   3.368200   9.329777   Topic3  -6.1625   0.4937
308  sentral   3.296331   8.554925   Topic3  -6.1841   0.5588
829    level   3.178595  15.807827   Topic3  -6.2205  -0.0915
146   bunga.   2.924723   5.675782   Topic3  -6.3037   0.8495

[206 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1598      1  0.961253  #adadikompas
2496      3  0.890069          #dxy
500       3  0.978276        #issue
501       3  0.978276          #isu
912       1  0.960737       #update
...     ...       ...           ...
136       3  0.162471        turun,
1389      1  0.857245        untung
15        1  0.263163         utang
15        2  0.614046         utang
15        3  0.175442         utang

[242 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

In [72]:
# Generating the best topics
top_topics = lda_model.print_topics(num_words=10)  # Displaying the top 10 keywords for each topics

# Create DataFrame
df_topics = pd.DataFrame(top_topics, columns=['Topic', 'Keywords'])

# Set topic as index
df_topics.set_index('Topic', inplace=True)

# Show df_topics
df_topics

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Keywords
Topic,
0,"0.034*""bunga"" + 0.034*""suku"" + 0.017*""bi"" + 0...."
1,"0.039*""suku"" + 0.038*""bunga"" + 0.016*""bi"" + 0...."
2,"0.075*""bunga"" + 0.074*""suku"" + 0.030*""bi"" + 0...."


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Topik 0:**

Fokus pada suku bunga, Bank Indonesia, kenaikan suku bunga, nilai tukar, dan saham. Topik ini mungkin berkaitan dengan kebijakan suku bunga Bank Indonesia dan dampaknya terhadap pasar saham dan nilai tukar rupiah.

**Topik 1:**

Juga berfokus pada suku bunga, Bank Indonesia, dan pasar, tetapi juga mencakup kebijakan, Federal Reserve (fed), dan Kredit Pemilikan Rumah (KPR). Topik ini mungkin berkaitan dengan kebijakan suku bunga dan pengaruhnya terhadap pasar keuangan, termasuk KPR.

**Topik 2:**

Fokus utamanya adalah suku bunga dan Bank Indonesia, dengan kata kunci seperti suku bunga acuan dan persentase. Topik ini mungkin berfokus pada kebijakan suku bunga Bank Indonesia dan pengumuman terkait suku bunga acuan.